In [1]:
#This file contains the functions necessary to the execution of the attack.

In [2]:
import sage.matrix.matrix_integer_dense_hnf as hnf

n=256
q=8380417

R.<xx>=QuotientRing(ZZ[x], ZZ[x].ideal(x**n+1))
Rq.<xxx>=QuotientRing(GF(q)[x], GF(q)[x].ideal(x**n+1))

In [3]:
def shift(L,j):
    temp=[0 for _ in range(256)]
    for _ in range(256):

        if _+j<256:
            temp[_+j]=L[_]
            
        if _+j>=256:
            temp[(_+j)%256]=-L[_]
            
    return temp

def modpm(x,a): 
    r=x%a
    if a%2==1:
        if r>a//2:
            r=r-a
            
    if a%2==0:
        if r>a//2:
            r=r-a
    return r

In [4]:
def s1gen():
    s1vec=[randint(-2,2) for _ in range(n)]
    return sum([s1vec[i]*xx^i for i in range(n)])

def faultyz1gen(s1,d):
    y1=sum([randint(-2**17+1,2**17)*xx^i for i in range(d)])

    cvec=[0]*n
    for loop in range(39):
        i=randint(0,n-1)
        while cvec[i]!=0:
            i=randint(0,n-1)
        e=randint(0,1)
        cvec[i]=((-2*e)+1)
    c=sum([cvec[i]*xx^i for i in range(n)])
    z1=y1+c*s1
    
    return (c,z1)

In [6]:
def attack_single_sig(d,e,bkz_size=25,complete_atk=False):
    s1=s1gen()
    (c,z)=faultyz1gen(s1,d)
    
    try:
        cinv=1/Rq(c.lift())
    
    except ZeroDivisionError:
        print("c not invertible")
        return 0

    print("Starting attack")
    
    t=cputime(subprocesses=True)
    
    latvec=[(cinv*xxx**i).lift().list() for i in range(d)]
    latvec=[(cinv*Rq(z.lift())).lift().list()] + latvec
    
    if complete_atk==True:
        
        nb_bloc=0
        key_guess=[0 for _ in range(n)]
        bloc_list=[]
        while (nb_bloc+1)*e-3*nb_bloc<n:
            
            bloc_list.append([_ for _ in range(n)][nb_bloc*e-3*nb_bloc:(nb_bloc+1)*e-3*nb_bloc+1])
            nb_bloc+=1
            
        bloc_list.append([_ for _ in range(n+1)][-e-1:])    
        
        print("Finding the key piece by piece..")
        for _ in bloc_list:
            print(str(_[0])+" to "+str(_[-2])+"..")
            M=Matrix(ZZ,latvec)
            M=Matrix(ZZ,M[::,_[0]:_[-1]])
            dim=M.nrows()
            diagq=Matrix(ZZ,[[0]*i + [q] + [0]*(e-i-1) for i in range(e)])
            M=M.stack(diagq)
    
            if bkz_size is None: 
                M=M.LLL()
        
            else: 
                M=M.BKZ(block_size=bkz_size)
        
            v=M[dim]
            
            if key_guess[_[0]]==v[0] and key_guess[_[1]]==v[1] and key_guess[_[2]]==v[2]:
                for i in range(e):
                    key_guess[_[i]]=v[i]
            else:
                for i in range(e):
                    key_guess[_[i]]=-v[i]
        
        s1=s1.list()
        ms1=[-_ for _ in s1]
        
        time=cputime(subprocesses=True)-t
        print("Attack time:",time)
        print("Recovered vector:",key_guess)
        print("Truncated key:", s1)
        print("Recovered vector ?= Truncated key: or -Truncated Key", key_guess==s1 or key_guess==ms1)
        
        return time,key_guess==ms1 or key_guess==s1
       
    else:
        M=Matrix(ZZ,latvec)
        M=Matrix(ZZ,M[::,:e] ) 
        dim=M.nrows()
        diagq=Matrix(ZZ,[[0]*i + [q] + [0]*(e-i-1) for i in range(e)])
        M=M.stack(diagq)
    
        if bkz_size is None: 
            M=M.LLL()
        
        else: 
            M=M.BKZ(block_size=bkz_size)
        
        v=M[dim]
        s1=s1.list()
        time=cputime(subprocesses=True)-t
        print("Attack time:",time)
        print("Recovered vector:",v)
        print("Truncated key:", s1[:e])
        print("Recovered vector ?= Truncated key: or -Truncated Key", (-v).list()==s1[:e] or (v).list()==s1[:e])
    
        return time,(-v).list()==s1[:e] or (v).list()==s1[:e]

In [17]:
def attack_two_sig(d,e,bkz_size=25,complete_atk=False):
    s1=s1gen()
    (c1,z1)=faultyz1gen(s1,d)
    (c2,z2)=faultyz1gen(s1,d)
    
    try:
        c1inv=1/Rq(c1.lift())
        c2inv=1/Rq(c2.lift())
    
    except ZeroDivisionError:
        print("c1 or c2 not invertible")
        return 0

    print("Starting attack")
    
    t=cputime(subprocesses=True)
    
    latvec1=[(c1inv*xxx**i).lift().list() for i in range(d)]
    latvec1=[(c1inv*Rq(z1.lift())).lift().list()] + latvec1
    latvec2=[(c2inv*xxx**i).lift().list() for i in range(d)]
    latvec2=[(c2inv*Rq(z2.lift())).lift().list()] + latvec2
    M1=matrix(ZZ,latvec1)
    M2=Matrix(ZZ,latvec2)
    M=span(Matrix(GF(q),M1)).intersection(span(Matrix(GF(q),M2))).basis()
    Mat=Matrix(ZZ,M)
    
    if complete_atk==True:
        nb_bloc=0
        key_guess=[0 for _ in range(n)]
        bloc_list=[]
        while (nb_bloc+1)*e-3*nb_bloc<n:
            
            bloc_list.append([_ for _ in range(n)][nb_bloc*e-3*nb_bloc:(nb_bloc+1)*e-3*nb_bloc+1])
            nb_bloc+=1
            
        bloc_list.append([_ for _ in range(n+1)][-e-1:])    
        
        print("Finding the key piece by piece..")
        for _ in bloc_list:
            print(str(_[0])+" to "+str(_[-2])+"..")
            M=Matrix(ZZ,Mat[::,_[0]:_[-1]])
            dim=M.nrows()
            #print("Subspace dimension on Fq:",dim)
            diagq=Matrix(ZZ,[[0]*i + [q] + [0]*(e-i-1) for i in range(e)])
            M=M.stack(diagq)
    
            if bkz_size is None: 
                M=M.LLL()
        
            else: 
                M=M.BKZ(block_size=bkz_size)
        
            v=M[dim]
            
            if key_guess[_[0]]==v[0] and key_guess[_[1]]==v[1] and key_guess[_[2]]==v[2]:
                for i in range(e):
                    key_guess[_[i]]=v[i]
            else:
                for i in range(e):
                    key_guess[_[i]]=-v[i]
        
        s1=s1.list()
        ms1=[-_ for _ in s1]
        
        time=cputime(subprocesses=True)-t
        print("Attack time:",time)
        print("Recovered vector:",key_guess)
        print("Truncated key:", s1)
        print("Recovered vector ?= Truncated key: or -Truncated Key", key_guess==s1 or key_guess==ms1)
        
        return time,key_guess==ms1 or key_guess==s1
       
    else:
    
        M=Matrix(ZZ,Mat)
        dim=M.nrows()
        M=Matrix(ZZ,M[::,:e] ) 
        print("Subspace dimension on Fq:",dim)
        diagq=Matrix(ZZ,[[0]*i + [q] + [0]*(e-i-1) for i in range(e)])
        M=M.stack(diagq)
    
        if bkz_size is None: 
            M=M.LLL()
        
        else: 
            M=M.BKZ(block_size=bkz_size)

        v=M[dim]

        time=cputime(subprocesses=True)-t
        print("Attack time:",time)
        print("Recovered vector:",v)
        print("Truncated key:", s1.lift().list()[:e])
        print("Recovered vector ?= Truncated key: or -Truncated Key", (-v).list()==s1.lift().list()[:e] or (v).list()==s1.lift().list()[:e])
        return time,(-v).list()==s1.lift().list()[:e] or (v).list()==s1.lift().list()[:e] 

In [34]:
def attack_many_sig(m,e,bkz_size=25,complete_atk=False):
    s1=s1gen()
    C=[]
    Cinv=[]
    Z=[]
    CinvZ=[]
    for _ in range(m):
        (c,z)=faultyz1gen(s1,n-1) 
        
        try:
            cinv=1/Rq(c.lift())
    
        except ZeroDivisionError:
            print("c not invertible")
            return 0
        
        C.append(c)
        Cinv.append(cinv)
        Z.append(z)
        CinvZ.append((Cinv[_]*Rq(Z[_].lift())))

    print("Starting attack ")
    
    t=cputime(subprocesses=True)
    
    LATVEC=[]

    Mat=Matrix(GF(q),[[(C[j]*xx**i)[n-1] for i in range(n)] for j in range(m) ])
    Vec=vector([Z[j][n-1] for j in range(m)])   
    
    if complete_atk==True:
        
        nb_bloc=0
        key_guess=[0 for _ in range(n)]
        bloc_list=[]
        while (nb_bloc+1)*e-3*nb_bloc<n:
            
            bloc_list.append([_ for _ in range(n)][nb_bloc*e-3*nb_bloc:(nb_bloc+1)*e-3*nb_bloc+1])
            nb_bloc+=1
            
        bloc_list.append([_ for _ in range(n+1)][-e-1:])    
        
        print("Finding the key piece by piece..")
        for _ in bloc_list:
            print(str(_[0])+" to "+str(_[-2])+"..")
            
            x0=Mat.solve_right(Vec)[_[0]:_[-1]]
            
            N=(Mat.right_kernel()).basis()
            N=Matrix(ZZ,N) 
            dim=N.nrows()
            N=Matrix(ZZ,N[::,_[0]:_[-1]]) 
            #print("Subspace dimension on Fq:",dim)
            diagq=Matrix(ZZ,[[0]*i + [q] + [0]*(e-i-1) for i in range(e)])
            N=N.stack(diagq)

            if bkz_size is None: 
                x0=vector(ZZ,x0)
                M = N.LLL()
                M=M[dim:,::]
                G = M.gram_schmidt()[0]
                target = -x0
                for i in reversed(range(G.nrows())):
                    c = ((target * G[i]) / (G[i] * G[i])).round()
                    target -= M[i] * c
            
                v=target    
     
            else: 
                x0=vector(ZZ,x0)
                M = N.BKZ(block_size=bkz_size)
                M=M[dim:,::]
                G = M.gram_schmidt()[0]
                target = -x0
                for i in reversed(range(G.nrows())):
                    c = ((target * G[i]) / (G[i] * G[i])).round()
                    target -= M[i] * c
            
                v=target
        
            if key_guess[_[0]]==v[0] and key_guess[_[1]]==v[1] and key_guess[_[2]]==v[2]:
                for i in range(e):
                    key_guess[_[i]]=v[i]
            else:
                for i in range(e):
                    key_guess[_[i]]=-v[i]
        s1=s1.list()
        ms1=[-_ for _ in s1]
        
        time=cputime(subprocesses=True)-t
        print("Attack time:",time)
        print("Recovered vector:",key_guess)
        print("Truncated key:", s1)
        print("Recovered vector ?= Truncated key: or -Truncated Key", key_guess==s1 or key_guess==ms1)
        
        return time,key_guess==ms1 or key_guess==s1
       
    else:
    
        x0=Mat.solve_right(Vec)[:e]
        N=(Mat.right_kernel()).basis()
        N=Matrix(ZZ,N) 
        dim=N.nrows()
        N=Matrix(ZZ,N[::,:e]) 
        print("Subspace dimension on Fq:",dim)
        diagq=Matrix(ZZ,[[0]*i + [q] + [0]*(e-i-1) for i in range(e)])
        N=N.stack(diagq)

        if bkz_size is None: 
            x0=vector(ZZ,x0)
            M = N.LLL()
            M=M[dim:,::]
            G = M.gram_schmidt()[0]
            target = -x0
            for i in reversed(range(G.nrows())):
                c = ((target * G[i]) / (G[i] * G[i])).round()
                target -= M[i] * c
            
            s0=target    
     
        else: 
            x0=vector(ZZ,x0)
            M = N.BKZ(block_size=bkz_size)
            M=M[dim:,::]
            G = M.gram_schmidt()[0]
            target = -x0
            for i in reversed(range(G.nrows())):
                c = ((target * G[i]) / (G[i] * G[i])).round()
                target -= M[i] * c
            
            s0=target
        
        Time=cputime(subprocesses=True)-t
        print("Attack time:",Time)
        print("Truncated key:", s1.lift().list()[:e])
        print("Recovered vector ?= Truncated key:", s0.list())
        print("Recovered vector ?= Truncated key:", (-s0).list()==s1.lift().list()[:e] or (s0).list()==s1.lift().list()[:e])

        return float(Time),(-s0).list()==s1.lift().list()[:e] or (s0).list()==s1.lift().list()[:e]